In [6]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymysql
import requests

# API details 
api_key="9vwTpZnV8T3jHXHKfpAU116yzoSYHMJJ8iimvq7w"
# Fetch data from API
url = f"https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key={api_key}"
data = requests.get(url).json()

# Connect using PyMySQL
conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="Laptop@321",
    database="tennis"
)
cursor = conn.cursor()

# Insert complexes and venues
for comp in data.get("complexes", []):
    cursor.execute("""
        INSERT INTO Complexes (complex_id, complex_name) 
        VALUES (%s, %s) 
        ON DUPLICATE KEY UPDATE complex_name = %s
    """, (comp["id"], comp["name"], comp["name"]))

    for v in comp.get("venues", []):
        cursor.execute("""
            INSERT INTO Venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                venue_name = VALUES(venue_name),
                city_name = VALUES(city_name),
                country_name = VALUES(country_name),
                country_code = VALUES(country_code),
                timezone = VALUES(timezone),
                complex_id = VALUES(complex_id)
        """, (
            v["id"],
            v["name"],
            v["city_name"],
            v["country_name"],
            v["country_code"],
            v["timezone"],
            comp["id"]
        ))

# Finalize and close
conn.commit()
cursor.close()
conn.close()
